## MFA System

### General points

- `FlodymArray` and `Stock` objects work without `MFASystem`

- Advantages of `MFASystem`:
  - Having all attributes in one namespace, including the "parent" dimension set.
  - Integrated data read-in
  - More export and plotting
  - Checks for `NaN` values and mass-balance

- Different ways / levels of integration for setup & data read-in
  - Shown here: suggested one

- Quite some code needed, only to show rough principle

### Attributes

- `dims` (`DimensionSet`)
- dictionaries:
  - `flows` (name: `Flow`)
  - `stocks` (name: `Stock`)
  - `processes` (name: `Process`*)
  - `parameters`  (name: `Parameter`)

### Three ingredients for MFA System

- System Definition
  - Which dimensions and processes are there?
  - Which flows, parameters, stocks are there?
    - Which dimensions do they have?
- Data input from files
  - dimension items (which years, regions, ...)
  - parameter values
- Compute routine

### Definition

In [ ]:
import flodym as fd

dimension_definitions = [
    fd.DimensionDefinition(letter="t", name="time", dtype=int),
    fd.DimensionDefinition(letter="p", name="product", dtype=str),
]

In [ ]:
parameter_definitions = [
    fd.ParameterDefinition(name="manufacturing", dim_letters=("t",)),
    fd.ParameterDefinition(name="manufacturing loss rate", dim_letters=("t",)),
    fd.ParameterDefinition(name="product shares", dim_letters=("p",)),
    fd.ParameterDefinition(name="product lifetimes", dim_letters=("p",)),
]

In [ ]:
process_names = [
    "sysenv",
    "manufacturing",
    "use"
]

In [ ]:
flow_definitions = [
    fd.FlowDefinition(from_process_name="sysenv", to_process_name="manufacturing", dim_letters=("t",)),
    fd.FlowDefinition(from_process_name="manufacturing", to_process_name="sysenv", dim_letters=("t",)),
    fd.FlowDefinition(from_process_name="manufacturing", to_process_name="use", dim_letters=("t", "p")),
    fd.FlowDefinition(from_process_name="use", to_process_name="sysenv", dim_letters=("t", "p")),
]

In [ ]:
stock_definitions = [
    fd.StockDefinition(
        name="use",
        process="use",
        dim_letters=("t", "p"),
        subclass=fd.InflowDrivenDSM,
        lifetime_model_class=fd.LogNormalLifetime,
    ),
]

In [ ]:
mfa_definition = fd.MFADefinition(
    dimensions=dimension_definitions,
    parameters=parameter_definitions,
    processes=process_names,
    flows=flow_definitions,
    stocks=stock_definitions,
)

### Data sources 

- Only give the locations

In [ ]:
dimension_files = {
    "time": "data/dimension_time.csv",
    "product": "data/dimension_product.csv",
}

# file contents as format example
print(open("data/dimension_product.csv", "r").read())

In [ ]:
parameter_files = {
    "manufacturing": "data/parameter_production.csv",
    "manufacturing loss rate": "data/parameter_manufacturing_loss_rate.csv",
    "product shares": "data/parameter_product_shares.csv",
    "product lifetimes": "data/parameter_product_lifetimes.csv",
}

# file contents as format example
print(open("data/parameter_product_shares.csv", "r").read())

### Compute routine: Write your own subclass

- Flodym provides `MFASystem` parent class (a pydantic `BaseModel`)
- Subclass should implement individual `compute` function
  - Has to be consistent with the flows, stocks, and parameters
- The rest is up to you (sub-functions etc)

In [ ]:
class SimpleMFA(fd.MFASystem):
    def compute(self):

        # manufacturing flows
        self.flows["sysenv => manufacturing"][...] = self.parameters["manufacturing"]
        self.flows["manufacturing => sysenv"][...] = self.flows["sysenv => manufacturing"] * self.parameters["manufacturing loss rate"]
        total_products = self.flows["sysenv => manufacturing"] - self.flows["manufacturing => sysenv"]
        self.flows["manufacturing => use"][...] = total_products * self.parameters["product shares"]

        # use stock
        self.stocks["use"].inflow[...] = self.flows["manufacturing => use"][...]
        self.stocks["use"].lifetime_model.set_prms(
            mean=self.parameters["product lifetimes"],
            std=0.5*self.parameters["product lifetimes"],
        )
        self.stocks["use"].compute()

        # end-of-life  flow
        self.flows["use => sysenv"][...] = self.stocks["use"].outflow

- notice the automatic flow naming (this is customizable) 

### Init, load & compute

In [ ]:
mfa_example = SimpleMFA.from_csv(
    definition=mfa_definition,
    dimension_files=dimension_files,
    parameter_files=parameter_files,
)

In [ ]:
mfa_example.compute()

In [ ]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

mfa_example.check_mass_balance()

### Sankey Plotting

In [ ]:
import flodym.export as fde

fig = fde.PlotlySankeyPlotter(mfa=mfa_example, exclude_processes=[]).plot()
fig.show()